In [1]:
import os
import sys
venv_path = os.environ['VIRTUAL_ENV']
sys.path.append(os.path.join(os.environ['VIRTUAL_ENV'], "lib\site-packages"))

## Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import folium
import pandas as pd
import random
from scipy import stats
%matplotlib inline

## Consts

In [3]:
PROVIDER_AND_ID = "provider_and_id"
YEAR = "accident_year"
LAT = "latitude"
LONG = "longitude"
X = "x"
Y = "y"
SEVERITY = "injury_severity_hebrew"
ROAD_SEGMENT_NAME = 'road_segment_name'
ROAD_SEGMENT = 'road_segment_id'
SEVERITY_DEAD = 'הרוג'
SEVERITY_HARD = 'פצוע קשה'
RELEVANT_KEYS_ANALYSIS = [PROVIDER_AND_ID, YEAR, ROAD_SEGMENT_NAME, ROAD_SEGMENT, LAT, LONG, X, Y, SEVERITY]

DEFAULT_ZOOM = 13

## Load data

In [4]:
csv_path = r"C:\Users\Asya\Downloads\involved_markers_hebrew.csv"
data = pd.read_csv(csv_path)

C:\Users\Asya\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# get only relevant data
filtered = data[RELEVANT_KEYS_ANALYSIS].drop_duplicates(subset=PROVIDER_AND_ID)

## Find outliers by zscore

In [6]:
def calc_outliers_for_segment(data, zscore_thresh, min_sample_size):
    coords = data[[X, Y]].dropna()
    
    # check if sample size is big enough for the statistic calculation
    if len(coords.drop_duplicates()) <= min_sample_size:
        return []
    
    # calculate zscores for each column
    zscores = stats.zscore(coords, axis = 0)
    zscores = np.nan_to_num(zscores)
    zscores = np.abs(zscores)
    # get indices to outliers
    outliers = np.argwhere(zscores > zscore_thresh)
    outlier_rows = np.unique(outliers[:,0])
    if len(outliers) > 0:
        outlier_rows = data.iloc[outlier_rows]
        return outlier_rows[RELEVANT_KEYS_ANALYSIS]
    return []

def get_outliers_by_z_scores(data, zscore_thresh, min_sample_size = 0):
    res = []
    for seg, rows in data.groupby(ROAD_SEGMENT):
        outliers = calc_outliers_for_segment(rows, zscore_thresh, min_sample_size)
        if len(outliers) > 0:
            res.append(outliers)
    return pd.concat(res)

## Pick threshold and minimum sample size

In [7]:
ZSCORE_THRESH = 5
MIN_SAMPLE_SIZE = 10

## Calculate outliers

In [8]:
outliers = get_outliers_by_z_scores(filtered, ZSCORE_THRESH, MIN_SAMPLE_SIZE)

In [9]:
outliers

,provider_and_id,accident_year,road_segment_name,road_segment_id,latitude,longitude,x,y,injury_severity_hebrew
513972,32012017141,2012,מחלף גנות - מחלף שפירים,10020.0,31.792094,35.163908,215611.0,633258.0,NaN
664530,32009074193,2009,מחלף גנות - מחלף שפירים,10020.0,31.800946,35.095771,209159.0,634244.0,NaN
779097,32011017777,2011,מחלף גנות - מחלף שפירים,10020.0,31.799720,35.185478,217654.0,634103.0,פצוע קל
923369,32009048869,2009,מחלף גנות - מחלף שפירים,10020.0,31.801540,35.104863,210020.0,634309.0,פצוע קל
1093617,32015020947,2015,מחלף גנות - מחלף שפירים,10020.0,31.792094,35.163908,215611.0,633258.0,פצוע קל
...,...,...,...,...,...,...,...,...,...
69011,12019077024,2019,כניסה לבאקה אל-גרבייה - צומת נרבתה,5740010.0,32.247190,35.069946,206775.0,683730.0,NaN
1864661,12019049379,2019,כניסה לבאקה אל-גרבייה - צומת נרבתה,5740010.0,32.247190,35.069946,206775.0,683730.0,פצוע קל
534824,32008026236,2008,צומת שלומי - צומת אדמית,8990016.0,33.119300,35.568447,253422.0,780498.0,פצוע קל
880697,12010059473,2010,צומת בדק - צומת ברקת,46130004.0,32.023163,34.945389,194977.0,658909.0,פצוע קל


## Visualize severe accident outlier

In [10]:
severe_outliers = outliers.loc[(outliers[SEVERITY] == SEVERITY_HARD) | (outliers[SEVERITY] == SEVERITY_DEAD)]
severe_outliers

,provider_and_id,accident_year,road_segment_name,road_segment_id,latitude,longitude,x,y,injury_severity_hebrew
1353688,12009028585,2009,מחלף דניאל - מחלף ענבה,10054.0,31.842993,34.985828,198757.0,638922.0,פצוע קשה
277398,12017023016,2017,כניסה לדרך האלוף עוזי נרקיס - מחלף אדומים,10872.0,31.826228,34.996430,199757.0,637061.0,פצוע קשה
671538,12018004430,2018,מחלף גהה - מחלף מורשה,40140.0,33.095556,35.105271,210192.0,777810.0,פצוע קשה
777873,12018049397,2018,מחלף גהה - מחלף מורשה,40140.0,33.095556,35.105271,210192.0,777810.0,פצוע קשה
1752036,12018048989,2018,מחלף גהה - מחלף מורשה,40140.0,33.095565,35.105281,210193.0,777811.0,פצוע קשה
186230,12018009696,2018,צומת נחל חדרה - צומת אור עקיבא,40210.0,33.095195,35.105164,210182.0,777770.0,פצוע קשה
771143,12009019226,2009,צומת גלילות - מחלף הכפר הירוק,50010.0,32.092972,35.104834,210047.0,666625.0,פצוע קשה
122028,12008012613,2008,מחלף מורשה - מחלף ירקון,50030.0,32.092963,35.104685,210033.0,666624.0,פצוע קשה
699958,12008025324,2008,מחלף מורשה - מחלף ירקון,50030.0,32.092963,35.104696,210034.0,666624.0,פצוע קשה
718282,12008008899,2008,מחלף מורשה - מחלף ירקון,50030.0,32.092963,35.104696,210034.0,666624.0,פצוע קשה


In [11]:
random_outlier = severe_outliers.sample(n=1)
random_outlier

,provider_and_id,accident_year,road_segment_name,road_segment_id,latitude,longitude,x,y,injury_severity_hebrew
304824,12018022384,2018,מחלף גולני - מחלף עיילבון,650110.0,32.471785,34.900583,190885.0,708669.0,הרוג


In [12]:
def plot_outlier(outlier_row):
    # create map and add accident to map
    coord = outlier_row[[LAT, LONG]]
    outlier_map = folium.Map(location=coord, zoom_start=DEFAULT_ZOOM)
    folium.Marker(coord).add_to(outlier_map)
    
    return outlier_map

In [13]:
plot_outlier(random_outlier)

## Save output

In [14]:
len(data), len(outliers)

(1909382, 567)

In [15]:
outliers.to_csv("D:\outliers.csv", header=True, index=False, encoding='utf-8')

In [16]:
updated_data = data[~data[PROVIDER_AND_ID].isin(outliers[PROVIDER_AND_ID])]
len(updated_data)

1907773

In [17]:
updated_data.to_csv("D:\involved_markers_hebrew_updated.csv", header=True, index=False, encoding='utf-8')

KeyboardInterrupt: 